In [1]:
import sys
sys.path.append('../..')
import numpy as np
import matplotlib.pyplot as plt
import random

# PyTorch
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import cuda
from torch.autograd import Variable
from torchvision.utils import make_grid
from torchvision.models.mobilenet import mobilenet_v2

# DLBio and own scripts
from DLBio.pytorch_helpers import get_device
import ds_ear_siamese
import transforms_data as td
from helpers import cuda_conv
import metrics as M
from siamese_network_train import Training
from ContrastiveLossFunction import ContrastiveLoss
from NN_Siamese import SiameseNetwork

In [2]:
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

def show_plot(epochs, value1, value2, label1, label2, plt_number):
    plt.figure(plt_number)
    plt.plot(epochs,value1, label=label1)
    plt.plot(epochs,value2, label=label2)
    plt.legend()
    plt.grid()

In [3]:
# Set Up All Configurations here
class Config():
    #1. Boolean ändern
    #2. FC-Layer auf Bild anpassen
    #3. LR auf 0,0005
    NN_SIAMESE = False
    USE_SAVED_MODEL = True
    dataset_dir = '../dataset/'
    # training_dir = "../data/ears/training/"
    # testing_dir = "../data/ears/testing/"
    train_batch_size = 16
    val_batch_size = 16
    test_batch_size = 16
    vis_batch_size = 8
    num_workers = 3
    
    EPOCHS= 30
    LEARNINGRATE = 0.001
    #WEIGHT_DECAY = 0.0

    TRESHOLD_VER = 0.8


In [15]:
def check(p):
    return (p>=0 and p<=1)
def percentage_split(num_classes, train_percentage, valid_percentage, test_percentage):
    '''
    This function return the amount of classes to be chosen, given the actual amount and the percentages for
    the different sets
    Arguments
    ---------
    num_classes:    Amount of classes in dataset
    train_percentage:   Percentage of one, percentual size of train dataset
    valid_percentage:   Percentage of one, percentual size of validation dataset
    test_percentage:   Percentage of one, percentual size of test dataset. Can be Zero

    Returns
    ---------
    The amount of classes for train, validation and test dataset
    '''
    assert round(train_percentage+valid_percentage+test_percentage, 1)==1.0, "Percentages do not add up to 1"
    assert check(train_percentage) and check(valid_percentage) and check(test_percentage)

    train = round(num_classes * train_percentage)
    if test_percentage == 0:
        valid = num_classes-train
        test = test_percentage
    else:
        valid = round( (num_classes*valid_percentage) )
        test = num_classes-train-valid
    return train,valid,test

def diff(first, second):
    ''' computes the list diff
    '''
    second = set(second)
    return [item for item in first if item not in second]

In [18]:
# define indicies to split Data
dset = ds_ear_siamese.get_dataset(data_path=Config.dataset_dir, transform_mode='size_only')
N = len(dset)
classes = [dset.class_to_idx[class_] for class_ in dset.classes]
num_classes = len(classes)
print(N)

# Get amount of classes per set
tr,va,te = percentage_split(num_classes=num_classes,
                            train_percentage=0.6, 
                            valid_percentage=0.3,
                            test_percentage=0.1
                            )
# create class separation
train_classes = random.sample(classes, tr)
classes = diff(classes, train_classes)
valid_classes = random.sample(classes, va)
classes = diff(classes, valid_classes)
test_classes = random.sample(classes, te)
classes = diff(classes, test_classes)

train_indices, val_indices, test_indices = [],[],[]
# go through
for i,(_, class_idx) in enumerate(dset.imgs):
    if class_idx in train_classes:
        train_indices.append(i)
        continue
    if class_idx in valid_classes:
        val_indices.append(i)
        continue
    if class_idx in test_classes:
        test_indices.append(i)
        continue


['alexander_bec', 'alina_sch', 'alissa_buh', 'beatrix_mah', 'clara_pau', 'clemens_blu', 'collin_sch', 'david_fau', 'falco_len', 'felix_mec', 'gregor_spi', 'hammam_als', 'janna_qua', 'jesse_kru', 'jule_dre', 'konrad_von', 'lars_fin', 'marcel_nim', 'marina_fri', 'mila_wol', 'mohammed_muh', 'moritz_bor', 'nils_loo', 'robert_kle', 'sarah_feh', 'tim_moe', 'yannik_obe']
2160
[3, 20, 15, 0, 7, 14, 9, 21, 16, 13, 1, 8, 17, 10, 19, 22] [24, 11, 12, 26, 2, 4, 18, 6] [25, 23, 5]
0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 1
81 1
82 1
83 1
84 1
85 1
86 1
87 1
88 1
89 1
90 1
91 1
92 1
93 1
94 1
95 1
96 1
97 1
98 1
99 1
100 1
101 1
102 1
103 1
104 1
105 1
1

In [ ]:
# definde data loader
# dl_train = ds_ear_siamese.get_dataloader(
train_dataloader = ds_ear_siamese.get_dataloader(
    data_path=Config.dataset_dir,
    indices=train_indices,
    batch_size=Config.train_batch_size,
    num_workers=Config.num_workers,
    transform_mode='siamese', # TODO switch to another transform?
    should_invert = False
)

val_dataloader = ds_ear_siamese.get_dataloader(
    data_path=Config.dataset_dir,
    indices=val_indices,
    batch_size=Config.val_batch_size,
    num_workers=Config.num_workers,
    transform_mode='siamese_valid_and_test',
    should_invert = False
)
# dl_test = ds_ear_siamese.get_dataloader(
test_dataloader = ds_ear_siamese.get_dataloader(
    data_path=Config.dataset_dir,
    indices=test_indices,
    batch_size=Config.test_batch_size,
    num_workers=Config.num_workers,
    transform_mode='siamese_valid_and_test',
    should_invert = False
)

vis_dataloader = ds_ear_siamese.get_dataloader(
    data_path=Config.dataset_dir,
    indices=train_indices,
    batch_size=Config.vis_batch_size,
    num_workers=Config.num_workers,
    transform_mode='siamese',
    should_invert = False
)

In [ ]:
# visualize some data....
dataiter = iter(vis_dataloader)

example_batch = next(dataiter)
concatenated = torch.cat((example_batch[0], example_batch[1]),0)
imshow(make_grid(concatenated))
print(example_batch[2].numpy())

In [ ]:
# Definde Model and load to device
if Config.NN_SIAMESE == False:
    if Config.USE_SAVED_MODEL:
        model = torch.load('./models/model_classification_MN_classifier_removed_96%.pt')
    else:
        model = mobilenet_v2(pretrained=True)
        model.features[0][0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        #model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=10)
        
        layers = []
        
        for layer in model.features[0]:
            layers.append(layer)
        model.features[0][0] = nn.ReflectionPad2d(1)
        model.features[0][1] = layers[0]
        model.features[0][2] = layers[1]
        model.features[0].add_module('3', layers[2])

        # list(model.features[2].children())[0].add_module('4', nn.Dropout2d(0.4))
        # list(model.features[3].children())[0].add_module('4', nn.Dropout2d(0.4))
        # list(model.features[6].children())[0].add_module('4', nn.Dropout2d(0.4))
        # list(model.features[8].children())[0].add_module('4', nn.Dropout2d(0.4))
        # list(model.features[10].children())[0].add_module('4', nn.Dropout2d(0.4))
        # list(model.features[12].children())[0].add_module('4', nn.Dropout2d(0.4))
        # list(model.features[14].children())[0].add_module('4', nn.Dropout2d(0.4))
        # list(model.features[16].children())[0].add_module('4', nn.Dropout2d(0.4))
        model.classifier = nn.Sequential(*list(model.classifier.children())[:-2])


else:
    model = SiameseNetwork()


device = get_device()
print(device)
model.to(device)

contrastive_loss_siamese = ContrastiveLoss(2.0)
optimizer_siamese = torch.optim.Adam(model.parameters(),lr = Config.LEARNINGRATE)

In [ ]:
#To Define which Layers we want to train
# for param in model.parameters():
#     param.requires_grad = False

# layers = list(model.children())[0]
# sub_layer = list(layers.children())
# unfreezed = [15,16,17,18]
# for u in unfreezed:
#     for param in sub_layer[u].parameters():
#         param.requires_grad = True

In [ ]:
# To show trainable parameters
from DLBio.pytorch_helpers import get_num_params

get_num_params(model,True)

In [ ]:
training = Training(model=model, optimizer=optimizer_siamese,train_dataloader=train_dataloader, val_dataloader=val_dataloader, loss_contrastive=contrastive_loss_siamese, nn_Siamese=Config.NN_SIAMESE, THRESHOLD=Config.TRESHOLD_VER)

# epochs, loss_history, val_loss_history, acc_history, val_acc_history = training(Config.EPOCHS)
# show_plot(epochs, loss_history, val_loss_history,'train_loss', 'val_loss',1)
# show_plot(epochs, acc_history, val_acc_history,'train_acc', 'val_acc', 2)

In [ ]:
# tn Bilder nicht gleich, Distanz größer als THRESH
# fp Bilder nicht gleich, Distanz kleiner als THRESH
# fn Bilder gleich, Distanz größer als THRESH
# tp Bilder gleich, Distanz kleiner als THRESH

def calc_test_label(thresh=Config.TRESHOLD_VER):
    '''
    This function processes the test dataloader and returns the true labels and the predicted labels (depending on a threshold)
    Arguments
    ---------
    thresh:     Threshold for "same-different" classification
                default is the Config set threshhold
                

    Returns
    ---------
    Two lists of same length as image tuples in test loader with labels 1 or 0
    '''
    ground_truth_label, prediction_label = [], []

    for data in test_dataloader:
        # use training class for data extraction
        label, output1, output2 = training.get_label_outputs(data)
        # extend labels of the ground truth
        ground_truth_label.extend(label.flatten().tolist())
        # Extend the distance-threshold prediction
        prediction_label.extend(M.batch_predictions_bin(output1, output2, thresh))
    # lists containing all image tuple labels or predictions
    return ground_truth_label, prediction_label

ground_truth, prediction = calc_test_label(Config.TRESHOLD_VER)
# get confusion matrix
cf = M.cf_matrix(ground_truth, prediction)

In [ ]:
# Set parameters for confusion_matrix plot
labels = ['True Pos','False Neg','False Pos','True Neg']
categories = ['Same', 'Different']

# plot matrix
M.make_confusion_matrix(cf,
                        group_names=labels,
                        categories=categories,
                        cbar=True
                        )

In [ ]:
# preparation for ROC
# define lists for the rates
tprs = []
fprs = []
# Set all Thresholds to be tested
threshholds = [x / 10 for x in list(range(1,12))]

for t in threshholds:
    ground_truth, prediction = calc_test_label(t)
    cf = M.cf_matrix(ground_truth, prediction)
    _,_,_,sensitivity,specificity = M.get_metrics(cf)

    tprs.append(sensitivity)
    fprs.append( (1 - specificity) )

show_plot(fprs, tprs, tprs, "ROC", "ROC2", 3)


In [ ]:
#model = torch.load('/Users/falcolentzsch/Develope/Bachelorthesis/Bachelorthesis/models/model.pt')

In [ ]:
#torch.save(model,'/nfshome/lentzsch/Documents/Bachelorarbeit/Bachelorthesis/models/model_MN_1.pt')